In [33]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, Normalizer, StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import TomekLinks
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from imblearn.under_sampling import RandomUnderSampler

In [2]:
os.chdir("C:\\Users\\Seijsener\\Downloads")

In [3]:
churnData = pd.read_csv("Customer-Churn.csv")

In [4]:
churnData

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No,Yes,No,No,No,No,Month-to-month,29.85,29.85,No
1,Male,0,No,No,34,Yes,Yes,No,Yes,No,No,No,One year,56.95,1889.5,No
2,Male,0,No,No,2,Yes,Yes,Yes,No,No,No,No,Month-to-month,53.85,108.15,Yes
3,Male,0,No,No,45,No,Yes,No,Yes,Yes,No,No,One year,42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,No,No,No,No,No,Month-to-month,70.70,151.65,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,Male,0,Yes,Yes,24,Yes,Yes,No,Yes,Yes,Yes,Yes,One year,84.80,1990.5,No
7039,Female,0,Yes,Yes,72,Yes,No,Yes,Yes,No,Yes,Yes,One year,103.20,7362.9,No
7040,Female,0,Yes,Yes,11,No,Yes,No,No,No,No,No,Month-to-month,29.60,346.45,No
7041,Male,1,Yes,No,4,Yes,No,No,No,No,No,No,Month-to-month,74.40,306.6,Yes


In [8]:
churnData.dtypes

gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

In [10]:
churnData['TotalCharges'] = pd.to_numeric(churnData['TotalCharges'], errors='coerce')

In [17]:
churnData.dtypes

gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
MonthlyCharges      float64
TotalCharges        float64
Churn                object
dtype: object

In [16]:
churnData.isna().sum()

gender               0
SeniorCitizen        0
Partner              0
Dependents           0
tenure               0
PhoneService         0
OnlineSecurity       0
OnlineBackup         0
DeviceProtection     0
TechSupport          0
StreamingTV          0
StreamingMovies      0
Contract             0
MonthlyCharges       0
TotalCharges        11
Churn                0
dtype: int64

examine the rows with missing values in the 'TotalCharges' column along with their neighboring rows, 

In [23]:
# Create a list to store indices of neighboring rows
neighboring_indices = []

# Get the indices of rows with missing values in the 'TotalCharges' column
missing_values_rows = churnData[churnData['TotalCharges'].isnull()].index

# Extend the neighboring_indices list with the indices before and after each missing value's index
for idx in missing_values_rows:
    neighboring_indices.extend([idx - 1, idx, idx + 1])

# Remove duplicate indices and indices that are out of bounds
neighboring_indices = sorted(set(neighboring_indices))
neighboring_indices = [idx for idx in neighboring_indices if idx >= 0 and idx < len(churnData)]

# Create a new DataFrame containing neighboring rows
neighboring_rows = churnData.iloc[neighboring_indices]

# Print the selected rows
print(neighboring_rows)


      gender  SeniorCitizen Partner Dependents  tenure PhoneService  \
487     Male              0      No         No      72          Yes   
488   Female              0     Yes        Yes       0           No   
489     Male              0      No        Yes       1          Yes   
752     Male              0     Yes        Yes       7          Yes   
753     Male              0      No        Yes       0          Yes   
754   Female              0     Yes         No       2          Yes   
935   Female              1     Yes         No       4          Yes   
936   Female              0     Yes        Yes       0          Yes   
937   Female              0      No         No       6          Yes   
1081    Male              1     Yes         No      46          Yes   
1082    Male              0     Yes        Yes       0          Yes   
1083    Male              0      No         No       2          Yes   
1339    Male              0     Yes         No      66          Yes   
1340  

filling the empty 'TotalCharges' rows by looking at the corresponding 'MonthlyCharges' values. One approach is to use a simple calculation to derive the 'TotalCharges' based on the 'MonthlyCharges'. Assuming 'TotalCharges' is the cumulative total amount charged until a specific time, you can fill the missing values by adding the corresponding 'MonthlyCharges' to the previous 'TotalCharges' (if available).

In [31]:
churnData.sort_index(inplace=True)

# Iterate over each row
for index, row in churnData.iterrows():
    # Check if 'TotalCharges' is missing
    if pd.isnull(row['TotalCharges']):
        # Calculate 'TotalCharges' based on the previous row's 'TotalCharges' and current 'MonthlyCharges'
        churnData.at[index, 'TotalCharges'] = churnData.at[index - 1, 'TotalCharges'] + row['MonthlyCharges']

# Print the updated DataFrame
print(churnData)

      gender  SeniorCitizen Partner Dependents  tenure PhoneService  \
0     Female              0     Yes         No       1           No   
1       Male              0      No         No      34          Yes   
2       Male              0      No         No       2          Yes   
3       Male              0      No         No      45           No   
4     Female              0      No         No       2          Yes   
...      ...            ...     ...        ...     ...          ...   
7038    Male              0     Yes        Yes      24          Yes   
7039  Female              0     Yes        Yes      72          Yes   
7040  Female              0     Yes        Yes      11           No   
7041    Male              1     Yes         No       4          Yes   
7042    Male              0      No         No      66          Yes   

     OnlineSecurity OnlineBackup DeviceProtection TechSupport StreamingTV  \
0                No          Yes               No          No         

In [32]:
churnData.isna().sum()

gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

In [30]:
# Get the indices of rows with missing values in the 'TotalCharges' column after filling
missing_values_rows_filled = churnData[churnData['TotalCharges'].isnull()].index

# Extend the neighboring_indices list with the indices before and after each missing value's index
for idx in missing_values_rows_filled:
    neighboring_indices.extend([idx - 1, idx, idx + 1])

# Remove duplicate indices and indices that are out of bounds
neighboring_indices = sorted(set(neighboring_indices))
neighboring_indices = [idx for idx in neighboring_indices if idx >= 0 and idx < len(churnData)]

# Create a new DataFrame containing neighboring rows after filling missing values
neighboring_rows_filled = churnData.iloc[neighboring_indices]

# Print the selected rows after filling missing values
print(neighboring_rows_filled)


      gender  SeniorCitizen Partner Dependents  tenure PhoneService  \
487     Male              0      No         No      72          Yes   
488   Female              0     Yes        Yes       0           No   
489     Male              0      No        Yes       1          Yes   
752     Male              0     Yes        Yes       7          Yes   
753     Male              0      No        Yes       0          Yes   
754   Female              0     Yes         No       2          Yes   
935   Female              1     Yes         No       4          Yes   
936   Female              0     Yes        Yes       0          Yes   
937   Female              0      No         No       6          Yes   
1081    Male              1     Yes         No      46          Yes   
1082    Male              0     Yes        Yes       0          Yes   
1083    Male              0      No         No       2          Yes   
1339    Male              0     Yes         No      66          Yes   
1340  

In [38]:
# Select the relevant features and scale them using StandardScaler.

features = ['tenure', 'SeniorCitizen', 'MonthlyCharges', 'TotalCharges']
X = churnData[features]
y = churnData['Churn']

scaler = StandardScaler()

X_scaled = scaler.fit_transform(X)

In [39]:
# Split the data into a training set and a test set.

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [41]:
# Fit a logistic regression model on the training data and check accuracy on the test data.

logistic_model = LogisticRegression()
logistic_model.fit(X_train, y_train)

y_pred = logistic_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.8069552874378992


In [43]:
# Check for imbalance in the dataset.

print("Class distribution:")
print(y.value_counts())

Class distribution:
Churn
No     5174
Yes    1869
Name: count, dtype: int64


In [48]:
# Use resampling strategies to balance the classes.

smote = SMOTE(random_state=42)
X_resampled_smote, y_resampled_smote = smote.fit_resample(X_train, y_train)

rus = RandomUnderSampler(random_state=42)
X_resampled_rus, y_resampled_rus = rus.fit_resample(X_train, y_train)

In [49]:
# Fit the model and check accuracy after resampling.

# Fit logistic regression model on SMOTE-resampled data
logistic_model_smote = LogisticRegression()
logistic_model_smote.fit(X_resampled_smote, y_resampled_smote)
y_pred_smote = logistic_model_smote.predict(X_test)
accuracy_smote = accuracy_score(y_test, y_pred_smote)
print("Accuracy after SMOTE:", accuracy_smote)

# Fit logistic regression model on RandomUnderSampler-resampled data
logistic_model_rus = LogisticRegression()
logistic_model_rus.fit(X_resampled_rus, y_resampled_rus)
y_pred_rus = logistic_model_rus.predict(X_test)
accuracy_rus = accuracy_score(y_test, y_pred_rus)
print("Accuracy after RandomUnderSampler:", accuracy_rus)

Accuracy after SMOTE: 0.7437899219304471
Accuracy after RandomUnderSampler: 0.7444996451383961


Based on the accuracies provided:

Original model (without balancing): Accuracy = 0.8069
Model after SMOTE (up-sampling): Accuracy = 0.7438
Model after RandomUnderSampler (down-sampling): Accuracy = 0.7445
The highest accuracy among the three models is achieved by the original model without balancing, with an accuracy of approximately 0.8069.

However, accuracy alone may not be the best metric to evaluate models, especially in the case of imbalanced datasets. It's essential to consider other metrics such as precision, recall, and F1-score, as well as the specific requirements of your application, to determine the best-performing model.